In [ ]:
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image


import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from glob import glob

from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output,State, no_update
import plotly.graph_objects as go
import plotly.express as px
import random
import base64

In [ ]:
def create_cam_visualizer(img_path,resnet,activation_layer_name,fully_connected_layer_name):
    # make a model to get output before flatten
    activation_layer = resnet.get_layer(activation_layer_name)

    # create a model object
    model = Model(inputs=resnet.input, outputs=activation_layer.output)

    # get the feature map weights
    final_dense = resnet.get_layer(fully_connected_layer_name)
    W = final_dense.get_weights()[0]

    app = JupyterDash(__name__)

    server = app.server
    app.layout = html.Div([
            dcc.Input(
                id="upload-image", 
                type="text", 
                placeholder="Upload Image and get heatmap",
                style={
                    'width': '100%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px'
                },
            ),
#             html.Div(id='output-prediction'),
            html.Div(id='output-prediction',children = [])
        ])
    
    @app.callback(Output('output-prediction', 'children'),
              Input('upload-image', 'value'))
    def prediction(img_path):
        try:
            if img_path is None:
                return no_update
            img_path = img_path

            # Load the image
            img = image.load_img(img_path, target_size=(224, 224))
            x = preprocess_input(np.expand_dims(img, 0))
            fmaps = model.predict(x)[0] # 7 x 7 x 2048

              # get predicted class
            probs = resnet.predict(x)
            classnames = decode_predictions(probs)[0]

            classname = classnames[0][1]
            pred = np.argmax(probs[0])

            # get the 2048 weights for the relevant class
            w = W[:, pred]

            # "dot" w with fmaps
            cam = fmaps.dot(w)

            # upsample to 224 x 224
            # 7 x 32 = 224
            cam = sp.ndimage.zoom(cam, (32, 32), order=1)

            fig = px.imshow(cam)
            children_2 = [
                html.Img(src=img),
                dcc.Graph(figure=fig),
                html.Div([
                    html.P('Major classes predicted are:-'),

                ]),
                html.Div([html.P(f"{classes}")  for classes in classnames])
            ]
            return children_2
        except:
            return None
    return app
    

In [ ]:
# add preprocessing layer to the front of VGG
resnet = ResNet50(input_shape=(224, 224, 3), weights='imagenet', include_top=True)

# view the structure of the model
# if you want to confirm we need activation_49
resnet.summary()




In [ ]:
for layer in resnet.layers:
    print(layer.name)

In [ ]:
image_path = r"C:\Users\gprak\Downloads\DiscUdemy Courses\bits-logo.png"
app = create_cam_visualizer(image_path,resnet,'conv5_block3_out','predictions')


In [ ]:
app.run_server(mode="external",port=8035)